# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import sys
sys.path.append('../../src')

import os
import logging

# Set logging level to suppress warnings
logging.getLogger().setLevel(logging.ERROR)
# Just disables the warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from nn.models import Models
from bots import BotBid
from bidding import bidding
from util import hand_to_str
from deck52 import random_deal
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies

INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding_same-5556000
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-binfo_same-5556000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/Jack/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\Models/lefty_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/dummy_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/righty_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/decl_nt-950000
INFO:tensorflow:Restoring parameters from ..\Models/lefty_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/dummy_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/righty_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/decl_suit-1278000


### Bidding through an entire auction for all four hands

Now we'll take a random deal and let the bot bid for each hand. We assume North deals and nobody is vulnerable.

This script was used to detect the reason for some strange bidding. But the error is fixed, so this should look normal now

In [3]:
hands = ['T8732.K7532.J.Q3', '4.AQJ98.KQ975.94', 'AQ.T6.A863.JT862', 'KJ965.4.T42.AK75']

In [4]:
bidder_bots = [BotBid([True, False], hand, models, sampler, i, 3, False) for i, hand in enumerate(hands)]

In [5]:
auction = ['PAD_START','PAD_START','PAD_START']  # since West deals we need 'PAD_START'

turn_i = len(auction) % 4 # whose turn is it to bid
step = 0

while not bidding.auction_over(auction):
    bid = bidder_bots[turn_i].bid(auction)
    # if step== 1:
    # bid.bid='X'
    if step== 3:
        bid.bid='2N'
    #if step== 5:
    #    bid.bid='2C'
    print(bid.to_dict())
    auction.append(bid.bid)
    turn_i = (turn_i + 1) % 4  # next player's turn
    step += 1
print(auction)

{'bid': '1S', 'who': 'NN', 'candidates': [{'call': '1S', 'insta_score': 0.988}], 'hcp': [10.1, 9.3, 9.3], 'shape': [2.6, 4.0, 3.3, 3.0, 2.8, 3.9, 3.2, 2.9, 2.7, 3.9, 3.3, 3.0]}
{'bid': 'PASS', 'who': 'NN', 'candidates': [{'call': 'PASS', 'insta_score': 0.999}], 'hcp': [7.3, 11.3, 14.3], 'shape': [1.7, 3.0, 4.4, 4.0, 1.3, 3.2, 4.3, 4.1, 5.1, 1.9, 3.1, 2.8]}
{'bid': '2H', 'who': 'NN', 'candidates': [{'call': '2H', 'insta_score': 1.0}], 'hcp': [8.2, 12.7, 6.8], 'shape': [3.3, 3.0, 2.7, 4.0, 5.0, 2.3, 2.6, 3.1, 3.4, 3.0, 2.7, 4.0]}
{'bid': '2N', 'who': 'NN', 'candidates': [{'call': 'PASS', 'insta_score': 0.992}], 'hcp': [11.5, 5.4, 12.2], 'shape': [5.5, 2.0, 2.8, 2.6, 3.3, 3.3, 3.4, 2.9, 2.0, 5.6, 3.0, 2.5]}
{'bid': 'PASS', 'who': 'Simulation', 'quality': 'Good', 'candidates': [{'call': 'PASS', 'insta_score': 0.884, 'expected_score': 49, 'adjustment': 44}, {'call': 'X', 'insta_score': 0.079, 'expected_score': 64, 'adjustment': 4}], 'samples': ['ATxx.8x.Jxx.T98x x.AKJT9xx.K98x.x Q8x.Qxx.AQx